In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import os
import random
import cv2
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Defining the directory path and listing the sub directories inside it.

In [ ]:
data_dir = "../input/doom-crossing"
sub_dir = os.listdir(data_dir)
sub_dir

# Plotting the image

In [ ]:
categories = ['animal_crossing', 'doom']

for category in categories:
    path = os.path.join(data_dir, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        plt.imshow(img_array)
        plt.show()
        break
    break

In [ ]:
print(img_array)

In [ ]:
img_array.shape

# Reshaping the image to size 50*50

In [ ]:
IMG_SIZE = 80
new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
plt.imshow(new_array)
plt.show()

# Preparing the training dataset by appending the images from the animal and doom directory

In [ ]:
training_data = []

def create_training_data():
    for category in categories:
        path = os.path.join(data_dir, category)
        category_count = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, category_count])
            except Exception as e:
                pass
        
create_training_data()       

In [ ]:
print(len(training_data))

# Shuffling the training data so that both doom and animal data be not be in sequence which will impact the model.

In [ ]:
random.shuffle(training_data)

# Separating features and label in the training dataset

In [ ]:
x = []
y = []

for features, label in training_data:
    x.append(features)
    y.append(label)

# Plotting random 50 images

In [ ]:
fix, axes = plt.subplots(nrows=10, ncols=5, figsize=(12,18))
axes = axes.flatten()
for i,ax in zip(range(50), axes):
    ax.imshow(x[i])
    ax.set_title(categories[y[i]])
    ax.axis("off")
    
plt.show()

# Converting the features into numpy array and one hot encoding the label

In [ ]:
x = np.array(x).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = to_categorical(y)
x.shape, y.shape

# Creating the model. Compiling and Training the model.

* Creating a Sequential model.
* Adding Flatten and Dense Layer in the model
* Adding an output layer and using the activation function = sigmoid
* Compiling the model. Keeping the cost function = binary_crossentropy and optimizer = adam

In [ ]:
model = models.Sequential()
model.add(layers.Flatten(input_shape=(80,80,1)))
model.add(layers.Dense(3200, activation='relu'))
model.add(layers.Dense(1600, activation='relu'))
model.add(layers.Dense(800, activation='relu'))
model.add(layers.Dense(400, activation='relu'))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(2,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

* Using ModelCheckPoint callback to save the model after every epoch.
* Using TensorBoard callback to visualize the model.
* Training the model using 33 percent data as the validation data and the call back functions

In [ ]:
NAME = "Animal-vs-Doom-Crossing"
checkpoint_path = "train_ckpt/cp.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
callbacks_list = ([checkpoint], [tensorboard])
model.fit(x, y, epochs=50, batch_size=110, shuffle=True, validation_split=0.33, callbacks=callbacks_list, verbose=1)

# Summarizing the model

In [ ]:
model.summary()

# Plotting the model

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)